# Modeling
(System) Modeling for control systems is figuring out the **relationship between the input(s) and output(s)** of a system.<br>
Usually this means that we want to write down the **Differential Equations (PDE/ODE)** that govern the underlying system we want to control.

Having a model of the system allows us to analyze the system, synthesis a control algorithm and test this on a simulation using said model.

During this lecture we mostly assume that we have a model at hand but coming up with one is a big part of being a control engineer thus we will give you a quick introduction to system modeling.

As mentioned in the lecture about modeling, *"All Models are wrong, some are useful"*. This is due to the fact that it is infeasible and/or impossible to perfectly capture the physics behind a system.
Thus the underlying model will always only be an approximation of reality and one needs to keep this in mind while creating and utilizing a model.


# Modeling Approaches
## General Steps
### 1. System Boundaries, Inputs and Outputs
First we must define the boundaries of the systems, which in turn defines the inputs and outputs of the system. During this step we define the complexity of the system.
1. **Boundaries:** The boundaries of the system define what quantities are worth keeping track off (i.e. become system states) and what quantities will become inputs or outputs.
This may sound trivial but it is an important first step of defining the model complexity.
For example while modeling a car it would not make sense to include a meteorological model of the winds affecting the car. Instead the winds will be introduced as an outside disturbance (input to the system).
2. **Inputs:** The inputs to a system define how the outside world affects the system itself. Inputs can be separated into controllable inputs (endogenous inputs) and not controllable inputs (exogenous inputs). The controllable inputs are in general the control inputs (for example the gas throttle from a car) and the uncontrollable inputs are usually disturbances. One has to be careful with the terminology as inputs can also be negative such as a leak in a water tank.
3. **Outputs:** The outputs of a system are the measurements or observations we make of the system. These can be separated in measured outputs (quantities we can measure directly such as the speed of a car) and performance outputs which we can't measure directly but would also like to control (for example the fuel efficiency of a car).

Some of the aspects can be controlled while deriving a model, meaning we can choose what quantities will be captured in the model, we can to some extend choose the inputs (usually the control inputs) to our system and we can control the outputs of our systems by using different sensors.

As an example on how we can choose those aspects, we look at Problem Set 1 Question 2:<br>
In this model we included the drag force, roll resistance, traction (engine model) and gravity as inputs to the system.
Another valid choice for inputs would be to ignore the drag force and only look at the roll resistance, traction and gravity.
If this is a good choice is another question but it would be a valid model of the system.
On the other hand one could make the model more complex and introduce the model that calculates the traction torque explicitly.
This would increase the scope of the system as new quantities such as temperatures and pressures become relevant besides to the car velocity and position become relevant.
This again would be valid but maybe it would be to complicated for the task at hand.
Lastly we can also change the outputs of the system. We are not limited to measuring the velocity of the car but we could also include a GPS device and measure the position explicitly.
If this is a good choice is questionable as the velocity is the only relevant quantity for cruise control.

### 2. Writing Down the Differential Equations
Once the system boundaries, inputs and outputs are defined the differential equations of the system need to be derived.
There are different approaches to doing this, but often this is done using so called storage variables (such as energy, temperature, mass, momentum, cash, etc) and how different inflows and outflows change these storage variables (mass flow, power heat flow, payments, etc.). Once these storage variables are defined and the inflows and outflows are known we can setup the differential equation using:
$$\frac{d}{dt}\text{storage} = \sum \text{inflows} - \sum \text{outflows}$$
Most known physical laws can be interpreted or rewritten as a variation of the storage approach. For example the well known Newton formula
$$
F = ma
$$
does not necessarily look like storage formulation in a classical sense but as you will learn in Mechanics III is a special case a momentum change.
This means that a lot of the time it is more convenient to use equations one has experience with to directly derive the differential equations.

What examples should I make here?

### 3. Writing Down in Standard Form
Every control system has different quantities to keep track of, different inputs/outputs and most importantly different orders of derivatives.
As this variety in models(variable names, derivative orders, inputs, output) would make it difficult to come up with unified theories that can be applied to all control systems it is important to write down the differential equations in a standard form, which looks like this:
$$
\dot x (t) = f(x(t), u(t))\\
y(t) = h(x(t), u(t))
$$
This ensures that all system have the same form and results are easily transferable.
In this formulation $f(x(t), u(t))$ is called the dynamics model and $h(x(t), u(t))$ is called the measurement model.
To end up in this formulation one first defines the inputs as a input vector $u(t)$, then define the outputs/measurements as an output vector $y(t)$ and lastly define the system states as state vector $x(t)$.

Example here





## Computing the Output Signal
What here?

## PS1 Exercise 3

In [103]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output, Image


class dynamic_model:
    
    def __init__(self):
        # constants
        self.m = 1                  #kg
        self.g = 9.81               #m/s^2
        self.l = 1                  #m
        self.cf = 0.5               #Nms/rad
        self.impulse_torque = 10    #Nm
        self.impulse_duration = 0.1 #s
        self.impulse_time = 3       #s
        self.t_end = 10             #s
        self.delta_t = 0.01         #s

    def dynamic_model_t(self, x, u, t):
        disturbance = 0
        if t > self.impulse_time and t < self.impulse_time + self.impulse_duration:
            disturbance = self.impulse_torque
        return np.array([x[1],
                        3*self.g/(2*self.l)*np.sin(x[0]) 
                        - 3*self.cf/(self.m*self.l**2)*x[1] 
                        + 3/(self.m*self.l**2)*u
                        + 3/(self.m*self.l**2)*disturbance])
    
    def euler(self, control_on, x_0, Kp, Kd):
        x = np.copy(x_0)
        x_history = np.copy(x_0)
        t = 0
        if control_on:
            while t < self.t_end:
                u = self.control(x, t, Kp, Kd)
                x = x + self.delta_t * self.dynamic_model_t(x, u, t)
                t += self.delta_t
                x_history = np.vstack((x_history, x))
            return x_history
        else:
            while t < self.t_end:
                x = x + self.delta_t * self.dynamic_model_t(x, 0, t)
                t += self.delta_t
                x_history = np.vstack((x_history, x))
            return x_history
    
    def control(self, x, t, Kp, Kd):
        # simple PD controller
        u = - Kp*x[0] - Kd*x[1]
        return u
    
model = dynamic_model()
def run_sim(control_on, K_p, K_d):
    x_0 = np.array([0, 0])
    x_history = model.euler(control_on, x_0, K_p, K_d)
    return x_history
    

In [104]:
# Initialize a display object
output = widgets.Output()

# Function to update the plot and title
def update_plot(control_off_checkbox, control_on_checkbox, Kp_slider, Kd_slider):
    with output:
        clear_output(wait=True)  # Clear the previous output
        # two subplots next to each other with given figure size:
        fig, axs = plt.subplots(1, 2, figsize=(20,5))
        
        if control_off_checkbox:
            x_out = run_sim(control_on=False, K_p=Kp_slider, K_d=Kd_slider)
            axs[0].plot(x_out[:,0], label='No Feedback', color='red')
            axs[1].plot(x_out[:,1], label='No Feedback', color='red')
        
        if control_on_checkbox:
            x_out = run_sim(control_on=True, K_p=Kp_slider, K_d=Kd_slider)
            axs[0].plot(x_out[:,0], label='With Feedback', color='blue')
            axs[1].plot(x_out[:,1], label='With Feedback', color='blue')
        
        # x lable with latex format of theta
        axs[0].set_xlabel(r'Time: t[s]')
        axs[0].set_ylabel(r'Angle: $\theta(t) [rad]$')
        axs[1].set_xlabel(r'Time: t[s]')
        axs[1].set_ylabel(r'Angular Velocity: $\dot\theta(t)[rad/s]$')
        for ax in axs:
            xticks = ax.get_xticks()  # Get current x-ticks
            ax.set_xticks(xticks)  # Fix the ticks first
            ax.set_xticklabels([f'{int(tick/100)}' for tick in xticks])  # Divide x-ticks by 100

        if control_off_checkbox or control_on_checkbox: # avoids legend error if none of the functions are selected
            axs[0].legend()
            axs[1].legend()
        axs[0].grid()
        axs[1].grid()
        plt.show()

# Create checkboxes to show controlled and uncontrolled plots
control_off_checkbox = widgets.Checkbox(value=True, description='Control off')
control_on_checkbox = widgets.Checkbox(value=False, description='Control on')

# Create a slider for Kp and Kd
Kp_slider = widgets.FloatSlider(value=0, min=-1, max=70, step=0.1, description="K_p gain")
Kd_slider = widgets.FloatSlider(value=0, min=-1, max=2, step=0.1, description="K_d gain")

# Arrange checkboxes in a vertical box
checkboxes_box = widgets.VBox([control_off_checkbox, control_on_checkbox])

# Arrange the checkboxes box and slider in a horizontal box
controls_box = widgets.HBox([checkboxes_box, widgets.VBox([Kp_slider, Kd_slider])])

# Link the widgets to the update_plot function
widgets.interactive_output(update_plot, {
    'control_off_checkbox': control_off_checkbox,
    'control_on_checkbox': control_on_checkbox,
    'Kp_slider': Kp_slider,
    'Kd_slider': Kd_slider
})

# Display the controls and the plot
display(controls_box, output)

Output()